# Week 2 Assignment using pandas and scikit learn

In [91]:
import pandas as pd 

### Load data

In [9]:
sales = pd.read_csv('kc_house_data.csv')
sales.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639


In [10]:
sales.columns

Index([u'id', u'date', u'price', u'bedrooms', u'bathrooms', u'sqft_living', u'sqft_lot', u'floors', u'waterfront', u'view', u'condition', u'grade', u'sqft_above', u'sqft_basement', u'yr_built', u'yr_renovated', u'zipcode', u'lat', u'long', u'sqft_living15', u'sqft_lot15'], dtype='object')

### 1. Selection and summary statistics: We found the zip code with the highest average house price. What is the average house price of that zip code?

In [39]:
avg_price_by_zip = sales.groupby(['zipcode']).mean()
print avg_price_by_zip.sort(columns='price', ascending=False)['price'].head(2)
print avg_price_by_zip.sort(columns='price', ascending=False)['price'].iloc[0]

zipcode
98039      2160606.600000
98004      1355927.082019
Name: price, dtype: float64
2160606.6


### 2. Filtering data: What fraction of the houses have living space between 2000 sq.ft. and 4000 sq.ft.?

In [44]:
sales_filtered = sales[(sales['sqft_living'] > 2000) & (sales['sqft_living'] < 4000)]
print sales_filtered.head(2)

            id             date   price  bedrooms  bathrooms  sqft_living  \
1   6414100192  20141209T000000  538000         3       2.25         2570   
10  1736800520  20150403T000000  662500         3       2.50         3560   

    sqft_lot  floors  waterfront  view  ...   grade  sqft_above  \
1       7242       2           0     0  ...       7        2170   
10      9796       1           0     0  ...       8        1860   

    sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
1             400      1951          1991    98125  47.7210 -122.319   
10           1700      1965             0    98007  47.6007 -122.145   

    sqft_living15  sqft_lot15  
1            1690        7639  
10           2210        8925  

[2 rows x 21 columns]


In [46]:
len(sales_filtered)/float(len(sales))

0.4215518437977143

### 3. Building a regression model with several more features: What is the difference in RMSE between the model trained with my_features and the one trained with advanced_features?

#### Select features

In [86]:
# all_features = sales.columns - ['price'] #remove output variable
my_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']
advanced_features = my_features + ['waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
print advanced_features

['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


#### Split data into train, test

In [115]:
import sklearn
#from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sales[my_features], sales['price'], test_size=0.2, random_state=0)
print X_train[0,:]
print X_test[0,:]
print y_train[0]

[  3.00000000e+00   1.00000000e+00   1.57000000e+03   5.51000000e+03
   1.00000000e+00   9.81150000e+04]
[  2.00000000e+00   1.50000000e+00   1.43000000e+03   1.65000000e+03
   3.00000000e+00   9.81250000e+04]
495000.0


In [126]:
advanced_X_train, advanced_X_test, advanced_y_train, advanced_y_test = train_test_split(sales[advanced_features], sales['price'], test_size=0.2, random_state=0)
print advanced_X_train[0,:] #same random_state or seed so same split
print advanced_X_test[0,:]

[  3.00000000e+00   1.00000000e+00   1.57000000e+03   5.51000000e+03
   1.00000000e+00   9.81150000e+04   0.00000000e+00   0.00000000e+00
   4.00000000e+00   7.00000000e+00   1.07000000e+03   5.00000000e+02
   1.94000000e+03   0.00000000e+00]
[  2.00000000e+00   1.50000000e+00   1.43000000e+03   1.65000000e+03
   3.00000000e+00   9.81250000e+04   0.00000000e+00   0.00000000e+00
   3.00000000e+00   7.00000000e+00   1.43000000e+03   0.00000000e+00
   1.99900000e+03   0.00000000e+00]


#### Run linear regression

In [107]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

In [124]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(X_test) - y_test) ** 2))
print("Root Mean squared error: %.2f"
      % np.sqrt(np.mean((regr.predict(X_test) - y_test) ** 2)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))

('Coefficients: \n', array([ -5.66006335e+04,   1.10870931e+04,   3.20408368e+02,
        -2.89499128e-01,  -2.41800204e+03,   6.17971044e+02]))
Mean squared error: 59538329742.70
Root Mean squared error: 244004.77
Variance score: 0.50


In [127]:
regr.fit(advanced_X_train, advanced_y_train)

LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

In [128]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(advanced_X_test) - advanced_y_test) ** 2))
print("Root Mean squared error: %.2f"
      % np.sqrt(np.mean((regr.predict(advanced_X_test) - advanced_y_test) ** 2)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(advanced_X_test, advanced_y_test))

('Coefficients: \n', array([ -3.72687641e+04,   4.35870226e+04,   1.16672560e+02,
        -2.62295351e-01,   2.64350610e+04,   2.16980566e+00,
         5.95624663e+05,   4.54348767e+04,   1.86372903e+04,
         1.22571768e+05,   5.75289860e+01,   5.91435733e+01,
        -3.56998516e+03,   8.44268043e+00]))
Mean squared error: 42200337367.03
Root Mean squared error: 205427.21
Variance score: 0.65
